# An initial look at raw PitchFX data

The goal of this notebook is to show you how to build and extract information from the SQLite3 *PitchFX* database using python codes and simple SQL commands.

Begin by importing the necessary libraries.

In [1]:
# imports
import commands
from IPython.display import display
import numpy as np
import pandas as pd
import sqlite3

## Build your PitchFX database

First let's specify a path and name for the database.

In [2]:
# specify database name and directory
dbname = "../Dat/example.db"

Next, we will build your personal *PitchFX* database using the codes provided in the Git repository (*scrape_pitchfx.py*, *load_pitchfx_mod.py*). The arguments you will need are the begin date, end date, name of your database, and whether you want an output or not. The format for the dates is a string following *MM-DD-YYYY*.

To run the commands, we will use the *subprocess* module, which will allow us to run commands as if we were on the command line. For this example, we will extract all data for games between April 1 and May 1, 2008. Note that this might take a while to build and requires internet access. You can also run these commands yourself in the command line.

In [3]:
# arguments for building your database
begin_date = "03-01-2008"
end_date = "05-01-2008"
prompt = 1

# build the command and run
comm = """python ../src/scrape_pitchfx.py %s %s %s %s""" %(begin_date, end_date, dbname, prompt)
print(comm)
output = commands.getoutput(comm)

python ../src/scrape_pitchfx.py 03-01-2008 05-01-2008 ../Dat/example.db 1


Now that we have build your database, let's start querying the database.

## Extract information from your PitchFX database

We'll practice extracting desired information from our Pitchfx database. This part is focused on using sql commands and is a way of performing a quality check on the database we created. Let's begin by connecting to your sqlite3 database.

In [4]:
# connect to the sqlite3 database
db = sqlite3.connect(dbname)
hdb = db.cursor()

First, let's print out the names of the tables in the database.

In [5]:
query = """SELECT name FROM sqlite_master WHERE type='table'"""
hdb.execute(query)
print(hdb.fetchall())

[(u'games',), (u'teams',), (u'stadiums',), (u'players',), (u'umpires',), (u'events',), (u'pitchfx',)]


Next, extract team information for the 'Oakland Athletics'.

In [6]:
query = """SELECT * 
    FROM teams 
    WHERE team_name='Oakland Athletics'"""
df = pd.read_sql_query(query, db)
display(df)

team_id          team_name team_abbr
0      133  Oakland Athletics       OAK

Next, extract player information for 'Barry Zito'.

In [7]:
query = """SELECT * 
    FROM players 
    WHERE player_first='Barry' 
        AND player_last='Zito'"""
df = pd.read_sql_query(query, db)
display(df)

player_id player_first player_last position bats throws       dob
0     217096        Barry        Zito        P    L      L  19780513

The *events* table contains all the events in a game, as well as the game situation during that event. Information includes a description of the event, the inning, the number of outs before and after the event, pitcher and batter ids, runner advancements, and score.

Extract events that involved 'Barry Zito' throwing a pitch. Sort events by game id and event id, and print out the first 20 entries.

We must use a **JOIN** statement to connect the name of the player to events he was involved in. Note that you can select just the information from the *events* table with **events.**\* command (and therefore do not need to also print out the associated information from the *player* table). We are using a subquery here to get only the information related to the specific player.

In [8]:
query = """SELECT events.*
    FROM events 
    JOIN players ON (events.pitcher_id=players.player_id)
    WHERE players.player_id=(SELECT player_id 
        FROM players 
        WHERE player_first='Barry' 
            AND player_last='Zito')
    ORDER BY game_id, event_id"""
df = pd.read_sql_query(query, db)
df.head(20)

game_id  event_id   event_description  inning  is_top_inning  pre_out  \
0    233769         7              Double       1              0        0   
1    233769         8             Pop Out       1              0        0   
2    233769         9              Single       1              0        1   
3    233769        10  Pickoff Attempt 2B       1              0        1   
4    233769        11  Pickoff Attempt 2B       1              0        1   
5    233769        12           Groundout       1              0        1   
6    233769        13              Flyout       1              0        2   
7    233769        18             Pop Out       2              0        0   
8    233769        19              Single       2              0        1   
9    233769        20  Pickoff Attempt 1B       2              0        1   
10   233769        21            Sac Bunt       2              0        1   
11   233769        22              Single       2              0        2   
12   233769        23  Pickoff Attempt 1B       2              0        2   
13   233769        24           Groundout       2              0        2   
14   233769        29           Groundout       3              0        0   
15   233769        30              Single       3              0        1   
16   233769        31              Single       3              0        1   
17   233769        32    Grounded Into DP       3              0        1   
18   233769        36              Single       4              0        0   
19   233769        37              Flyout       4              0        0   

    post_out  pitcher_id  batter_id  pre_1b  post_1b  pre_2b  post_2b  pre_3b  \
0          0      217096     279577       0        0       0   279577       0   
1          1      217096     444843       0        0  279577   279577       0   
2          1      217096     461314       0        0  279577   461314       0   
3          1      217096     116999       0        0  461314   461314       0   
4          1      217096     116999       0        0  461314        0       0   
5          2      217096     116662       0        0       0        0       0   
6          3      217096     431145       0        0       0        0       0   
7          1      217096     425766       0        0       0        0       0   
8          1      217096     455088       0   455088       0        0       0   
9          1      217096     207267  455088   455088       0        0       0   
10         2      217096     207267  455088        0       0   455088       0   
11         2      217096     279577       0   279577  455088        0       0   
12         2      217096     444843  279577   279577       0        0       0   
13         3      217096     444843  279577        0       0        0       0   
14         1      217096     461314       0        0       0        0       0   
15         1      217096     116999       0   116999       0        0       0   
16         1      217096     116662  116999   116662       0   116999       0   
17         3      217096     431145  116662        0  116999        0       0   
18         0      217096     425766       0   425766       0        0       0   
19         1      217096     455088  425766   425766       0        0       0   

    post_3b  post_home  pre_home_score  post_home_score  pre_away_score  \
0         0          0               0                0               0   
1         0          0               0                0               0   
2         0     279577               0                1               0   
3         0          0               1                3               0   
4         0     116999               1                3               0   
5         0          0               3                3               0   
6         0          0               3                3               0   
7         0          0               3                3               0   
8        

Next, find the games that 'Barry Zito' pitched in. Print out the total number of games pitched in during the time period you previously specified when building your database (in order of game id), and print out the first 5 games.

Again, we want to use a subquery to get only the information when the given player is pitching. Additionally, we use **DISTINCT** so that there are no repeated games (since there will very likely be multiple pitches thrown in by a player in a given game).

In [9]:
query = """SELECT DISTINCT game_id
    FROM events
    WHERE pitcher_id=(SELECT player_id
        FROM players
        WHERE player_first='Barry'
            AND player_last='Zito')
    ORDER BY game_id"""
df = pd.read_sql_query(query, db)
print("Number of games: %s" %(df.shape[0]))
df.head()

Number of games: 6


game_id
0   233769
1   233847
2   233914
3   233970
4   234063

Now let's try the same query but with a **JOIN** statement. Note that the output is the exact same as the previous query without using a join.

In [10]:
query = """SELECT DISTINCT game_id
    FROM events
    JOIN players ON (events.pitcher_id=players.player_id)
    WHERE pitcher_id=(SELECT player_id
        FROM players
        WHERE player_first='Barry'
            AND player_last='Zito')
    ORDER BY game_id"""
df = pd.read_sql_query(query, db)
print("Number of games: %s" %(df.shape[0]))
df.head()

Number of games: 6


game_id
0   233769
1   233847
2   233914
3   233970
4   234063

Next, let's look at joining multiple tables. Look at the *stadium*, *games*, and *events* tables. Print the games, game date, and stadiums that 'Barry Zito' pitched in.

In [11]:
query = """SELECT DISTINCT games.game_id, games.date, stadiums.stadium_name
    FROM games
    JOIN stadiums ON (games.stadium_id=stadiums.stadium_id)
    JOIN events ON (games.game_id=events.game_id)
    WHERE events.pitcher_id=(SELECT player_id
        FROM players
        WHERE player_first='Barry'
            AND player_last='Zito')
    ORDER BY date"""
df = pd.read_sql_query(query, db)
display(df)

game_id      date    stadium_name
0   233769  20080331  Dodger Stadium
1   233847  20080406     Miller Park
2   233914  20080411       AT&T Park
3   233970  20080416       AT&T Park
4   234063  20080422     Chase Field
5   234134  20080427       AT&T Park

Next, let's extract all information on all the pitches thrown by 'Barry Zito' by looking at the *pitchfx* table, looking at the first 5 pitches.

The *pitchfx* table contains all information about the trajectory of the ball from the pitcher's hand to the plate. The join of this table with the *events* table is on the *cur_event* key. Note that each pitch is not an event (new events occur when something happens in the field). Therefore, for each pitch, the current event reveals the situation in the field at the time of the pitch.

Note that *events* must be joined with *pitchfx* over two variables (to make the join distinct). Event ids are reset for each game, so join must also be done over game id. Additionally, the **DISTINCT ** is a precaution so that there are no duplicates in the result, as it is likely that the tables are not perfect.

In [12]:
query = """SELECT DISTINCT pitchfx.* 
    FROM pitchfx
    JOIN events ON (pitchfx.game_id=events.game_id
        AND pitchfx.cur_event=events.event_id)
    WHERE events.pitcher_id=(SELECT player_id
        FROM players
        WHERE players.player_first='Barry'
            AND players.player_last='Zito')
    ORDER BY game_id, pitch_num"""
df = pd.read_sql_query(query, db)
df.head()

game_id  pitch_num  at_bat      time  cur_event      description outcome  \
0   233769         30       4  134206.0          7             Ball       B   
1   233769         31       4  134217.0          7    Called Strike       S   
2   233769         32       4  134228.0          7             Foul       S   
3   233769         33       4  134247.0          7  In play, no out       X   
4   233769         37       5  134336.0          8             Ball       B   

   pre_balls  post_balls  pre_strike     ...        vz0     ax      ay  \
0          0           1           0     ...     -4.167 -4.395  28.973   
1          1           1           0     ...     -7.852 -1.776  29.654   
2          1           1           1     ...     -2.479  3.593  22.606   
3          1           1           2     ...     -5.803 -1.431  30.550   
4          0           1           0     ...     -2.421 -3.625  31.828   

       az  break_y  break_angle  break_length  spin_dir  spin_rate  pitch_type  
0 -13.125     23.7         16.5           3.9   192.990   2278.601          FC  
1  -9.818     23.7          8.9           3.2   184.540   2620.543          FF  
2 -23.886     23.7         -8.3           9.2   156.564   1243.876          CH  
3  -8.798     23.7         12.1           2.8   183.501   2732.740          FF  
4 -12.756     23.6         18.5           3.8   190.573   2300.700          FF  

[5 rows x 36 columns]

Now that we're extracted some raw table information, let's look extracting other types of information. For this, we check a relief pitcher. We'll start by looking at 'Arthur Rhodes' and the games, days, and stadiums that he appeared in.

In [13]:
query = """SELECT DISTINCT games.game_id, games.date, stadiums.stadium_name
    FROM games
    JOIN stadiums ON (games.stadium_id=stadiums.stadium_id)
    JOIN events ON (events.game_id=games.game_id)
    WHERE events.pitcher_id=(SELECT player_id
        FROM players
        WHERE player_first='Arthur'
            AND player_last='Rhodes')"""
df = pd.read_sql_query(query, db)
display(df)

game_id      date              stadium_name
0   233962  20080415              Safeco Field
1   234009  20080418  Angel Stadium of Anaheim
2   234039  20080420  Angel Stadium of Anaheim
3   234054  20080422              Safeco Field
4   234085  20080424              Safeco Field
5   234125  20080426              Safeco Field
6   234163  20080429         Progressive Field
7   234187  20080501         Progressive Field

Compared to the game log from *Baseball Reference*, the results look plausible. The number of appearances match and the general stadiums where those appearances occurred seem to match.

Next, we'll look at the number of pitches he's thrown in each appearence using sql's **COUNT** command. Note the use of the **GROUP BY** command, which is going to group the number of pitches by game. Also, note that in this case, we need to use **DISTINCT** because there are likely to be duplicates in the table (this a problem that has not been addressed in the creation of the tables).

In [14]:
query = """SELECT pitchfx.game_id, COUNT(DISTINCT pitch_num)
    FROM pitchfx
    JOIN events ON (pitchfx.game_id=events.game_id
        AND pitchfx.cur_event=events.event_id)
    WHERE events.pitcher_id=(SELECT player_id
        FROM players
        WHERE players.player_first='Arthur'
            AND players.player_last='Rhodes')
    GROUP BY pitchfx.game_id"""
df = pd.read_sql_query(query, db)
display(df)

game_id  COUNT(DISTINCT pitch_num)
0   233962                         11
1   234009                         17
2   234039                          5
3   234054                         13
4   234085                         14
5   234125                          4
6   234163                          6
7   234187                         24

Fortunately, the number of pitches thrown matches up with what is found on *Baseball Reference*.

Finally, let's clean up by closing the database connection.

In [15]:
# clean up
hdb.close()

Hopefully this is enough information to help your begin your PitchFX analysis!